# 의사록 데이터 크롤링

In [2]:
pip install tika

  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32888 sha256=b2ad79c85673f8cfcf5cbdd2b62ae02f184e0cc9ab246065c5dc8a31e842ea4c
  Stored in directory: c:\users\student\appdata\local\pip\cache\wheels\ec\2b\38\58ff05467a742e32f67f5d0de048fa046e764e2fbb25ac93f3
Successfully built tika
Note: you may need to restart the kernel to use updated packages.


In [12]:
import requests
from bs4 import BeautifulSoup
from tika import parser
import pandas as pd
from IPython.display import clear_output
import re

In [49]:
# 의사록 Crawling

import requests
from bs4 import BeautifulSoup
from tika import parser
import pandas as pd
from IPython.display import clear_output

start_page = 1
current_page = start_page
end_page = 31

# 32페이지는 링크따와서 수작업
host = 'http://www.bok.or.kr/'
result_df = pd.DataFrame(columns=['date', 'text'])

while True:
    URL = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(current_page)
    r = requests.get(URL)
    sp = BeautifulSoup(r.text, 'html.parser')
    results = sp.select('div.fileGoupBox ul')
    
    for i, result in enumerate(results):
        pdf_date = re.search('[0-9]{4}\.[0-9]{1,2}\.[0-9]{1,2}', sp.select('span.titlesub')[i].text).group()
        
        pdf_temp = requests.get(host + result.select('li')[1].select('a')[0].attrs['href'])

        with open('bok.pdf', 'wb') as f:
            f.write(pdf_temp.content)
        
        parsed = parser.from_file('bok.pdf')
        temp_text = str(parsed['content']).strip()
        temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
        result_df = pd.concat([result_df, temp_df])
    
    clear_output(wait=True)
    print('크롤링 {}% 진행 중...'.format(int((current_page/end_page)*100)))
        
    if current_page == end_page:
        clear_output(wait=True)
        print('데이터 크롤링 완료 ! bok_datas.csv에서 확인 가능')
        result_df.to_csv('bok_datas.csv')
        break
    current_page += 1

데이터 크롤링 완료 ! bok_datas.csv에서 확인 가능


In [24]:
# 32 페이지 수작업 부분
URL = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex=32'
r = requests.get(URL)
sp = BeautifulSoup(r.text, 'html.parser')
results = sp.select('div.fileGoupBox ul')

for i, result in enumerate(results):
    pdf_date = re.search('[0-9]{4}\.[0-9]{1,2}\.[0-9]{1,2}', sp.select('span.titlesub')[i].text).group()

    pdf_temp = requests.get(host + result.select('li')[1].select('a')[0].attrs['href'])

    with open('bok.pdf', 'wb') as f:
        f.write(pdf_temp.content)

    parsed = parser.from_file('bok.pdf')
    temp_text = str(parsed['content']).strip()
    temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])

In [25]:
temp_df

,date,text
0,2005.6.9,130.1.17.56-minute12.hwp\n\n\n- 1 -\n\n金融通貨委員會...


In [52]:
result_df = pd.concat([result_df, temp_df])

In [ ]:
result_df

In [13]:
# 안돌림
for i, result in enumerate(results):
    if i==0:
        continue
    
    pdf_date = re.search('[0-9]{4}\.[0-9]{1,2}\.[0-9]{1,2}', sp.select('span.titlesub')[i].text).group()

    pdf_temp = requests.get('http://www.bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085858&fileSn=1')

    with open('bok.hwp', 'wb') as f:
        f.write(pdf_temp.content)

    parsed = parser.from_file('bok.hwp')
    temp_text = str(parsed['content']).strip()
    temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
    


2020-07-24 16:26:00,934 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [28]:
pdf_temp = requests.get('http://www.bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085858&fileSn=1')
pdf_date = '2005.5.12'
with open('bok.hwp', 'wb') as f:
    f.write(pdf_temp.content)

parsed = parser.from_file('bok.hwp')
temp_text = str(parsed['content']).strip()
temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
temp_df

,date,text
0,2005.5.12,6.12.금통위 \n\n金融通貨委員會 議事錄 \n\n2005년도 제10차 회의 \n...


In [20]:
import pandas as pd
bok_df = pd.read_csv('bok_datas.csv')

In [31]:
bok_df.to_csv('bok_datas.csv')

In [29]:
bok_df = pd.concat([bok_df, temp_df])

In [ ]:
result_df.to_csv('bok_datas.csv', encoding='utf-8')

In [33]:
import pandas as pd
bok_df = pd.read_csv('bok_datas.csv', index_col=0)

In [35]:
bok_df.tail() # 앞부분 다름 index 299 양식 다름 - 여러 개 있음 index 300~ 양식 다름

,date,text
0,2005.8.11,130.1.17.56-minute17.hwp\n\n\n- 1 -\n\n金融通貨委員會...
0,2005.7.7,130.1.17.56-minute14.hwp\n\n\n- 1 -\n\n金融通貨委員會...
0,2005.6.23,130.1.17.56-minute13.hwp\n\n\n- 1 -\n\n金融通貨委員會...
0,2005.6.9,130.1.17.56-minute12.hwp\n\n\n- 1 -\n\n金融通貨委員會...
0,2005.5.12,6.12.금통위 \n\n金融通貨委員會 議事錄 \n\n2005년도 제10차 회의 \n...


In [34]:
bok_df

,date,text
0,2020.6.24,2020년도 제14차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\...
0,2020.6.11,2020년도 제13차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\...
0,2020.5.28,2020년도 제12차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\...
0,2020.5.14,2020년도 제11차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\...
0,2020.4.16,2020년도 제9차\n\n금융통화위원회(임시) 의사록\n\n한 국 은 행\n\n\n...
...,...,...
0,2005.8.11,130.1.17.56-minute17.hwp\n\n\n- 1 -\n\n金融通貨委員會...
0,2005.7.7,130.1.17.56-minute14.hwp\n\n\n- 1 -\n\n金融通貨委員會...
0,2005.6.23,130.1.17.56-minute13.hwp\n\n\n- 1 -\n\n金融通貨委員會...
0,2005.6.9,130.1.17.56-minute12.hwp\n\n\n- 1 -\n\n金融通貨委員會...


# 의사록 Section 분리

In [36]:
import re
import numpy as np
def section_split(bok_text):
    bok_text = bok_text.replace('\n', ' ')
    s2_ls = []
    s3_ls = []
    section2 = re.finditer('(?<=원 토의내용)((?!심의결과).)*', bok_text)
    for s2 in section2:
        s2_ls.append(s2.group())
    section3 = re.finditer('(?<=심의결과)[^１]*', bok_text)
    for s3 in section3:
        s3_ls.append(s3.group())
    return ' '.join(s2_ls), ' '.join(s3_ls)


split_text = list(map(section_split, bok_df['text']))

In [37]:
len(split_text)

312

In [38]:
s2_ls = []
s3_ls = []

for doc in split_text:
    s2_ls.append(doc[0])
    s3_ls.append(doc[1])

In [39]:
bok_df['section2'] = s2_ls

In [40]:
bok_df['section3'] = s3_ls

In [41]:
bok_df[['date', 'section2','section3']].to_csv('bok_split_section.csv', encoding='utf-8')

In [42]:
bok_df.isnull().sum()

date        0
text        0
section2    0
section3    0
dtype: int64

In [43]:
bok_df.iloc[30]['section2']

''

In [44]:
bok_df.iloc[30]['text']

'2018년도 제23차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\n\n- 1 -\n\n1. 일 자 2018년 12월 6일(목)\n\n2. 장 소 금융통화위원회 회의실\n\n3. 출석위원 이 주 열 의 장(총재)\n\n이 일 형 위 원\n\n조 동 철 위 원\n\n고 승 범 위 원\n\n신 인 석 위 원\n\n윤 면 식 위 원(부총재)\n\n임 지 원 위 원\n\n4. 결석위원 없 음\n\n5. 참 여 자 장 호 현 감 사 임 형 준 부총재보\n\n허 진 호 부총재보 신 호 순 부총재보\n\n정 규 일 부총재보 서 봉 국 외자운용원장\n\n이 환 석 조사국장 손 욱 경제연구원장\n\n신 운 금융안정국장 박 종 석 통화정책국장\n\n이 상 형 금융시장국장 이 승 헌 국제국장\n\n장 정 석 금융통화위원회실장 김 현 기 공보관\n\n성 광 진 의사팀장\n\n6. 회의경과\n\n<의안 제39호 - 대부업 등의 등록 및 금융이용자 보호에 관한 법률 시행령\n\n제9조제3항제1호의 연체이자율에 관한 규정 폐지규정(안)>\n\n(１) 담당 부총재보가 대부업 등의 등록 및 금융이용자 보호에 관한 법률 시\n\n행령 개정으로 한국은행법 제11조에 따른 금융기관에 대한 연체이자율 상한을\n\n금융위원회에서 정하게 됨에 따라 관련 규정을 폐지하고자 한다는 내용의 제안설\n\n명을 하였음.\n\n(２) 심의결과\n\n원안대로 가결\n\n\n\n- 2 -\n\n의결사항\n\n대부업 등의 등록 및 금융이용자 보호에 관한 법률 시행령 제9조제3항\n\n제1호의 연체이자율에 관한 규정 을 붙임과 같이 폐지한다.\n\n<붙임> 대부업 등의 등록 및 금융이용자 보호에 관한 법률 시행령 제9조\n\n제3항제1호의 연체이자율에 관한 규정 폐지규정(안)'

In [46]:
import pandas as pd
bok_datas = pd.read_csv('bok_split_section.csv',encoding='utf-8', index_col = 0)
# bok_datas = bok_datas[['date', 'section2', 'section3']]

In [35]:
# 확인용 안 돌림
for i in bok_datas.iloc:
    if type(i['section2']) != type(np.nan):
        if '정부측' in i['section2']:
            pass
#             print(i['section2'])
            
    else:
        print(i['date'])
        print(i['section3'])

In [36]:
test_str = bok_datas['section2'][50]

# 의사록 데이터 전처리

In [37]:
import random
bok_datas['section3'][random.randint(0,310)]

In [48]:
# Section 2

import re

for se2 in bok_datas.iloc:
    if type(se2['section2']) != type(np.nan):
        se2.section2 = se2.section2.strip()
        se2.section2 = re.sub('- [0-9]{1,2} -', '', se2.section2) # 페이지번호 삭제
        se2.section2 = re.sub('\([３４５６７８９0-9 ]*\)', '', se2.section2) # Section 분리기준 삭제
        se2.section2 = re.sub('\([가-힣]*\)', '', se2.section2) #가나다기호 삭제

# '토의결론.*' # 맨 끝 제거
# '관련부서는 ((?!보고하였음).)* 보고하였음\.'
# '일부 위원은'
# se2.replace('  ', '')        

# bok_datas

In [49]:
# Section 3

for se3 in bok_datas.iloc:
    if type(se3['section3']) != type(np.nan):
        se3.section3 = se3.section3.strip()
        se3.section3 = re.sub('\(생략\)', '', se3.section3) # 생략기호 삭제
        se3.section3 = re.sub('\<붙임\>', '', se3.section3) # 붙임기호 삭제
        se3.section3 = re.sub('개정\(안\)', '', se3.section3) # 무의미 단어 삭제
        se3.section3 = re.sub('원안대로 가결', '', se3.section3) # 무의미 단어 삭제
        se3.section3 = re.sub('의결문 작성.가결', '', se3.section3) # 무의미 단어 삭제
        se3.section3 = re.sub('의결사항', '', se3.section3) # 무의미 단어 삭제
        se3.section3 = re.sub('- [0-9]{1,2} -', '', se3.section3) # 페이지번호 삭제
        
# '앞서의 .* 였음.'

# bok_datas

In [52]:
bok_datas

,date,section2,section3,doc
0,2020.6.24,관련부서는 금번 보고서 작성의 기본방향과 위원협의회에서 논의된 주요 내용 ...,한국은행법 제96조 제1항에 의거 거시 금융안정상황에 대한 평가보고서를 붙...,관련부서는 금번 보고서 작성의 기본방향과 위원협의회에서 논의된 주요 내용 ...
0,2020.6.11,관련부서는 본 안건과 관련하여 위원협의회 등에서 논의된 내용을 다음과 같 ...,한국은행법 제96조 제1항에 의거 통화신용정책 수행상황에 대한 보고서를 붙...,관련부서는 본 안건과 관련하여 위원협의회 등에서 논의된 내용을 다음과 같 ...
0,2020.5.28,전일 개최된 동향보고회의에서는 다음과 같은 토의가 있었음. 경제전망(2020.5...,"앞서의 토의결과를 반영하여 다음과 같이 통화정책방향에 대한 의결문을 작 성, 가결...",전일 개최된 동향보고회의에서는 다음과 같은 토의가 있었음. 경제전망(2020.5...
0,2020.5.14,관련부서는 본 안건과 관련하여 5월 11일 위원협의회에서 논의된 내용을 다 음과 ...,2020년 5월 18일부터 적용할 한국은행의 금융기관에 대한 금융중개지원대출...,관련부서는 본 안건과 관련하여 5월 11일 위원협의회에서 논의된 내용을 다 음과 ...
0,2020.4.16,"제안설명에 이어 관련부서에서는 본 안건에 대해 세 차례(4월 7일, 4월 9일,...",｢금융안정특별대출제도 신설 및 운용방안｣을 붙임과 같이 의결한다. ｢금융...,"제안설명에 이어 관련부서에서는 본 안건에 대해 세 차례(4월 7일, 4월 9일,..."
...,...,...,...,...
0,2005.8.11,"｢국내외 경제동향｣과 관련하여, 일부 위원은 소비, 수출, 건설투자가 견조한 증...","(다만, 김태동 위원은 이에 대해 명백히 반대의사를 표시하고 콜금리 목 ...","｢국내외 경제동향｣과 관련하여, 일부 위원은 소비, 수출, 건설투자가 견조한 증..."
0,2005.7.7,"｢국내외 경제동향｣과 관련하여, 일부 위원은 최근 이란의 새 대통령 취임 이후 ...","(다만, 김태동 위원은 이에 대해 명백히 반대의사를 표시하고 ...","｢국내외 경제동향｣과 관련하여, 일부 위원은 최근 이란의 새 대통령 취임 이후 ..."
0,2005.6.23,일부 위원은 3월말 8.3조원이었던 총액한도대출 잔액이 4∼5월중 다소 늘 어 5...,2005년 3/4분기 한국은행의 금융기관에 대한 총액한도...,일부 위원은 3월말 8.3조원이었던 총액한도대출 잔액이 4∼5월중 다소 늘 어 5...
0,2005.6.9,"｢국내외 경제동향｣과 관련하여, 일부 위원은 최근 소비재판매액 증가세가 다소 둔...",｢통화정책방향｣을 다음과 같이 결정한다. □ 실물경제는 수출...,"｢국내외 경제동향｣과 관련하여, 일부 위원은 최근 소비재판매액 증가세가 다소 둔..."


In [50]:
# 합치기

temp = []

for se in bok_datas.iloc:
    
    if type(se.section2) == type(np.nan):
        se.section2 = ''
    if type(se.section3) == type(np.nan):
        se.section3 = ''
        
    temp.append(se['section2'] + se['section3'])
    
bok_datas['doc'] = temp

In [53]:
bok_datas[['date', 'doc']].set_index('date').to_csv('bok_final.csv') # 13117 kb
bok_datas[['date', 'doc']].set_index('date').to_json('bok_final.json') # 24516 kb

In [2]:
import pandas as pd
datas = pd.read_json('bok_final.json')

# 의사록 문장분류

In [14]:
import pandas as pd
datas = pd.read_csv('bok_final.csv')
len(datas)

312

In [15]:
# datas.iloc[15]['doc']

In [16]:
import numpy as np
import re
for doc in datas.iloc:
    if type(doc['doc']) != type(np.nan):
        doc['doc'] = re.sub('([0-9a-zA-Z]{1,100}\.[0-9a-zA-Z]{0,100}%?)', '', doc['doc']) # 소수점 전처리
        doc['doc'] = re.sub(' [가-힣]{1}\.', '', doc['doc']) # 가, 나, 다 기호 전처리

In [17]:
sents = []
for data in datas.iloc:
    temp = data['doc'].split('.')
    for s in temp:
        s = s.strip()
        for i in range(3):
            s = s.replace('  ', ' ')
        if len(s) < 2:
            continue
        
        sents.append(s)

In [18]:
len(sents)

38109

In [20]:
# sents